In [ ]:
from operator import mul
import urllib2
import numpy as np
import tensorflow as tf
import os
import cv2

In [ ]:
def from_buffer(im_str, sz, dtype_dat):
    f1 = np.fromstring(im_str, dtype=dtype_dat)
    return f1.reshape(sz)    



In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt


### For home robot

im_shape = (480, 640, 3)
im_sz = reduce(mul, im_shape)
ROBOT_URL = 'http://localhost:8889/robot/%d/%d/%d/%d/%d/%d/'



def get_images():
    response=urllib2.urlopen('http://localhost:8889/camera')
    left = from_buffer(response.read(im_sz), im_shape, np.uint8)
    right = from_buffer(response.read(im_sz), im_shape, np.uint8)
    response.close()
    return left, right


episode_save_location = '/data/robotics/pick_episodes_2/'
pyrobo_path = "/home/karthik/code/pyrobo/"

left_im, right_im = get_images()
plt.imshow(left_im)


### To be run for office robot


In [ ]:
col_sz = (480, 640, 3)
dep_sz = (480, 640)
pnt_sz = (480, 640, 3)
cad_sz = (480, 640, 3)
fcam_sz = (288, 352, 3)


col_dtype = np.uint8
dep_dtype = np.uint16
cad_dtype = np.uint8
pnt_dtype = np.float32


ROBOT_URL = 'http://robot.local:8888/robot/%d/%d/%d/%d/%d/%d/'

def get_front_camera():
    response=urllib2.urlopen('http://localhost:8891/camera')
    fc = from_buffer(response.read(288*352*3), fcam_sz, np.uint8)
    response.close()
    return fc


def get_images():
    response=urllib2.urlopen('http://robot.local:8888/camera')
    col = from_buffer(response.read(reduce(mul, col_sz)), col_sz, col_dtype)
    pnt = from_buffer(response.read(reduce(mul, pnt_sz)*4), pnt_sz, pnt_dtype)
    cad = from_buffer(response.read(reduce(mul, cad_sz)), cad_sz, cad_dtype)
    dep = from_buffer(response.read(reduce(mul, dep_sz)*2), dep_sz, dep_dtype)
    response.close()
    return col, pnt, cad, dep, get_front_camera()


episode_save_location = '/data/robotics/auto_episodic_data/'
pyrobo_path = '/Users/kk/code/pyrobo/'

col, pnt, cad, dep, fnt = get_images()
plt.imshow(cad)

In [ ]:
import json
def get_ps3_out():
    response=urllib2.urlopen('http://localhost:8888/get')
    js, buttons = json.loads(''.join(response.readlines()))
    pressed_buttons = []
    for (button, state) in js.items():
        if state > 0.5:
            pressed_buttons.append(button)
        if state < -0.5 and button == '1':
            pressed_buttons.append('0')
            
    for (button, state) in buttons.items():
        if state > 0.5:
            pressed_buttons.append(button)
    return pressed_buttons

motor_state = {0:1500, 1:1500, 2:1500, 3:1500, 4:1500, 5:1500}

import time
def move_robot():
    url = ROBOT_URL
    ms = [l[1] for l in sorted(motor_state.items(), key=lambda x: x[0])]
    call_url = url%tuple(ms)
    print call_url
    time.sleep(0.1)
    response = urllib2.urlopen(call_url)
    return response.readlines()

def update_state(motor_id, update_direction):
    motor_state[motor_id] = motor_state[motor_id] + 50*update_direction
    move_robot()
    
move_robot()    

 

#### Init model and model prediction

In [ ]:


def motor_direction_from_model(left_im, right_im, motor_state):
    
    out_pred = sess.run(tf.nn.softmax(net.layers['out']), { net.left_img:resize_to_model(left_im).reshape((1,224,224,3)),
        net.right_img: resize_to_model(right_im).reshape((1,224,224,3)), net.motor_state: motor_state} )
    choice = np.argmax(out_pred)
    motor = choice/2
    direction = choice%2*2-1
    print(motor, direction)
    return (motor, direction)


# def load_models(save_location=None):
#     global saver
#     net = PurchasePredNet()
#     if save_location is None:
#         net.load('/data/image/models/VGG_imagenet.npy', sess, True)
#     else:
#         if saver is None:
#             saver = tf.train.Saver()
#         saver.restore(sess, save_location)
#     return net

def motor_2_label(motor, direction):
    return 2*motor+(direction+1)/2

def motorstate_arr(motor_state):
    return np.array([float(motor_state[i]) for i in range(6)], dtype=np.float32)

def load_all_training_data():
    train_files = [os.path.join(episode_save_location, f)for f in os.listdir(episode_save_location) 
                   if f.endswith('.tfrecords')]
    img_cnt = 0
    for train_fl in train_files:
        img_cnt+=data.TFDataReader(train_fl).count()
    train_inp = np.zeros([2,img_cnt, 224, 224, 3], dtype=np.uint8)
    train_labels = np.zeros([img_cnt, ], dtype=np.int32)
    train_states = np.zeros([img_cnt, 6], dtype=np.float32)
    idx = 0
    for train_fl in train_files:
        reader = data.TFDataReader(train_fl)
        fds = reader.readall(( ["left_image", data.IMG_TYP],
                            ["right_image",  data.IMG_TYP],
                            ['motor', data.INT_TYP],
                            ['direction', data.INT_TYP],
                            ['motor_state',  data.INT_TYP]))
        for fd in fds:
            train_inp[0, idx, ...] = resize_to_model(fd['left_image'])
            train_inp[1, idx, ...] = resize_to_model(fd['right_image'])
            motor = int(fd['motor'][0])
            direction = int(fd['direction'][0])
            train_states[idx,:] = np.array(fd['motor_state'])
            label = motor_2_label(motor, direction)
            train_labels[idx] = label
            idx+=1
    
    return (train_inp, train_labels, train_states, img_cnt)

def retrain_models():
    global saver
    train_inp, train_labels, train_states, total_samples = load_all_training_data()
    for step in range(1000):
        idxes = np.random.randint(0, total_samples-1, size=BATCH_SIZE)
        left_im = train_inp[0, idxes, ...]
        right_im = train_inp[1, idxes, ...]
        labels_x = train_labels[idxes]
        states = train_states[idxes, ...]
        [_, lv] = sess.run([train, loss], {labels_pl: labels_x, 
                                                    net.left_img:left_im,
                                                    net.right_img: right_im,
                                                  net.motor_state : states})
        if step%100 == 0:
            print("step: %d, loss: %f"%(step,lv))
    if saver is None:
        saver = tf.train.Saver()
        saver.save(sess, episode_model_save_location + '/episode_model_%d'%current_episode)
    return sess.run( loss, {labels_pl: train_labels[:100], 
                                                    net.left_img:train_inp[0,  :100,...],
                                                    net.right_img: train_inp[1,  :100, ...],
                                                   net.motor_state: train_states[:100,:]})


## Run script

In [ ]:
from tftools import data
import sys
saver = None
episode_model_save_location = '/data/robotics/models/auto_episodic'

sys.path.append(pyrobo_path + '/network_models/')
sys.path.append(pyrobo_path + '../pytools/src/')


In [ ]:
import trainer
sess = tf.Session()

with open(episode_save_location + '/episode', 'r') as f:
    current_episode = int(f.readline().strip())


# reload(trainer)
tr = trainer.OfficeTrainer(sess, episode_model_save_location,
                          episode_save_location, current_episode)

In [ ]:
dat = tr.get_data_to_save()

In [ ]:
len(dat)

In [ ]:
plt.imshow(dat[0][1])

In [ ]:
tr.retrain_model(4, steps=100, print_every=10)

In [ ]:
from tftools import data
import sys
saver = None
episode_model_save_location = '/data/robotics/models/auto_episodic'

sys.path.append(pyrobo_path + '/network_models/')
sys.path.append(pyrobo_path + '../pytools/src/')
from multicam_net import PurchasePredNetOffice

sess = tf.Session()
with open(episode_save_location + '/episode', 'r') as f:
    current_episode = int(f.readline().strip())




In [ ]:
saver = tf.train.Saver()

In [ ]:
?saver.restore

In [ ]:
motorstate_arr(motor_state)

In [ ]:
with open(episode_save_location + '/episode', 'r') as f:
    current_episode = int(f.readline().strip())

writer = data.TFDataWriter(episode_save_location + "/episode_%d.tfrecords"%current_episode)

try:
    while True:
        time.sleep(0.3)
        pressed = get_ps3_out()        
        for press in pressed:
            if press in button_motor_map or press == USE_MODEL_BUTTON:
#                 left_im, right_im = get_images()
                col, pnt, cad, dep, fnt = get_images()
                if press == USE_MODEL_BUTTON:
                    pass
#                   motor, direction = net.predict(left_im, right_im,  np.array([[float(motor_state[i]) for i in range(6)]], dtype=np.float32))
#                     motor, direction = net.predict(sess, cad, fnt, dep, motorstate_arr(motor_state))
                else:
                    motor, direction = button_motor_map.get(press,[None, None])
                    writer.add( 
                                [['motor', [motor], data.INT_TYP],
                                ['direction', [direction], data.INT_TYP],
                                ['motor_state', motorstate_arr(motor_state) , data.FLT_TYP]])


                if motor is None:
                    continue
                update_state(motor, direction)
finally:
    writer.close()
    motor_state = {0:1500, 1:1500, 2:1500, 3:1500, 4:1500, 5:1500}
    move_robot()
    with open(episode_save_location + '/episode', 'w') as f:
        f.write("%d"% (int(current_episode)+1))
    print current_episode
#     retrain_models()
    

In [ ]:
retrain_models()

### Retraining of models after every episode

In [ ]:
from multicam_net import PurchasePredNetOffice
net = PurchasePredNetOffice()

In [ ]:
sess = tf.Session()

In [ ]:
BATCH_SIZE=8
labels_pl = tf.placeholder(shape=(None,), dtype=tf.int32)
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=net.layers['out'], labels=labels_pl))
optimizer = tf.train.AdagradOptimizer(0.001)
train = optimizer.minimize(loss)
sess.run(tf.global_variables_initializer())

In [ ]:
net.load(, sess, True)

In [ ]:
for step in range(100):
    idxes = np.random.randint(0, img_cnt-1, size=BATCH_SIZE)
    main_im = train_inp[0, idxes, ...]
    front_im = train_inp[1, idxes, ...]
    labels_x = train_labels[idxes]
    depth_x = train_depths[idxes, ...]
    states = train_states[idxes, ...]
    
    [_, lv] = sess.run([train, loss], {labels_pl: labels_x, 
                                        net.main_img:main_im,
                                        net.front_img: front_im,
                                        net.depth : depth_x,
                                        net.motor_state : states})
    if step%10 ==0:
        print "Step %s, Loss %f"%(step, lv)


In [ ]:
saver = tf.train.Saver()

In [ ]:
saver.save(sess, '/data/robotics/models/auto_episodic/episode_model_%d'%4)

In [ ]:
from tftools import data
import sys
saver = None
episode_model_save_location = '/data/robotics/models/auto_episodic'

sys.path.append(pyrobo_path + '/network_models/')
sys.path.append(pyrobo_path + '../pytools/src/')
import multicam_net

def load_all_training_data():
    train_files = [os.path.join(episode_save_location, f)for f in os.listdir(episode_save_location) 
                   if f.endswith('.tfrecords')]
    img_cnt = 0
    for train_fl in train_files:
        img_cnt+=data.TFDataReader(train_fl).count()
    train_inp = np.zeros([2,img_cnt, 224, 224, 3], dtype=np.uint8)
    train_labels = np.zeros([img_cnt, ], dtype=np.int32)
    train_states = np.zeros([img_cnt, 6], dtype=np.float32)
    train_depths = np.zeros([img_cnt, 112 * 112], dtype=np.float32)
    idx = 0
    for train_fl in train_files:
        reader = data.TFDataReader(train_fl)
        fds = reader.readall(( ["main", data.IMG_TYP],
                            ["front",  data.IMG_TYP],
                            ["depth",  data.NPY_TYP],
                            ['motor', data.INT_TYP],
                            ['direction', data.INT_TYP],
                            ['motor_state',  data.FLT_TYP]))
        
        for fd in fds:
            train_inp[0, idx, ...] = multicam_net.resize_to_model(fd['main'])
            train_inp[1, idx, ...] = multicam_net.resize_to_model(fd['front'])
            train_depths[idx, ...] = multicam_net.resize_to_model(fd['depth'], (112, 112)).reshape((112*112))
            motor = int(fd['motor'][0])
            direction = int(fd['direction'][0])
            train_states[idx,:] = np.array(fd['motor_state'])
            label = motor_2_label(motor, direction)
            train_labels[idx] = label
            idx+=1
    
    return (train_inp, train_labels, train_states, train_depths, img_cnt)


In [ ]:
(train_inp, train_labels, train_states, train_depths, img_cnt) = load_all_training_data()

In [ ]:
train_depths.shape

In [ ]:
vdict = dict([(v.name,v) for v in tf.global_variables()])

vdict['out/weights:0'].eval(sess)

### PS3 only control

In [ ]:

try:
    while True:
        time.sleep(0.3)
        pressed = get_ps3_out()
        for press in pressed:
            motor, direction = button_motor_map.get(press,[None, None])
            if motor is None:
                continue
            update_state(motor, direction)
finally:
    motor_state = {0:1500, 1:1500, 2:1500, 3:1500, 4:1500, 5:1500}
    move_robot()


In [ ]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [ ]:
saver = tf.train.Saver()

saver.restore(sess, '/data/robotics/models/pick_episodes/25000_iters')

In [ ]:
import cv2
def rs(arr):
    return cv2.resize(arr, (224, 224)).reshape((1, 224, 224, 3))
left_im, right_im = [rs(x) for x in get_images()]

out_pred = sess.run(tf.nn.softmax(net.layers['out']), { net.left_img:left_im,
                                                    net.right_img: right_im} )

print(out_pred)
choice = np.argmax(out_pred)

motor = choice/2
direction = choice%2*2-1
print(motor, direction)
update_state(motor, direction)
# def extract_motor_direction(choice)

In [ ]:
example_out = tf.train.Example(features=tf.train.Features(feature={
                'img_sz': _int32_feature((480, 640, 3))}))


In [ ]:
img_sz = example_out.features.feature['img_sz'].int64_list.value

In [ ]:
img_sz[0],img_sz[1], img_sz[2]

In [ ]:
def _int32_feature(value):
    return tf.train.Feature(int32_list=tf.train.Int32List(value=value))


In [ ]:
from tftools import data as tfdata

In [ ]:
p = np.uint8

In [ ]:
%timeit col.tostring()

In [ ]:
%timeit col.tobytes()

In [ ]:
import cPickle


In [ ]:
%timeit cPickle.dumps(col.dtype)

In [ ]:
cPickle.dumps(col.dtype)

In [ ]:
%timeit cPickle.loads(cPickle.dumps(col.dtype))

### implementing spatial softmax

In [ ]:
net.load('/data/models/VGG_imagenet.npy', sess, True)

In [ ]:
import cv2
im = cv2.resize(cad,(224,224))

In [ ]:
im.shape

In [ ]:
l_conv_2 = sess.run([net.layers['l_conv2_2']], {net.left_img:im.reshape(1,224,224,3)})

In [ ]:
plt.imshow(l_conv_2[0][0, :, :, 1:4])

In [ ]:
convout = l_conv_2[0][0, :, :, 0]

In [ ]:
err = np.array([[1,0.5], [0.3, 0.7]])
tf.nn.softmax(err).eval()

In [ ]:
tf.nn.softmax(net.layers['l_conv2_2'],dim=1).shape

In [ ]:
plt.imshow(col)

In [ ]:
num_rows = 112
num_cols = 112
x_map = np.empty([num_rows, num_cols], np.float32)
y_map = np.empty([num_rows, num_cols], np.float32)

for i in range(num_rows):
    for j in range(num_cols):
        x_map[i, j] = (i - num_rows / 2.0) / num_rows
        y_map[i, j] = (j - num_cols / 2.0) / num_cols

x_map = tf.convert_to_tensor(x_map)
y_map = tf.convert_to_tensor(y_map)

x_map = tf.reshape(x_map, [num_rows * num_cols])
y_map = tf.reshape(y_map, [num_rows * num_cols])

features = tf.reshape(tf.transpose(net.layers['l_conv2_2'], [0,3,1,2]),
                          [-1, 128, num_rows*num_cols])

softmax = tf.nn.softmax(features)

fp_x = tf.reduce_sum(tf.multiply(x_map, softmax), 2)
fp_y = tf.reduce_sum(tf.multiply(y_map, softmax), 2)

fp = tf.reshape(tf.concat(axis=1,values=[fp_x, fp_y]), [-1, 128*2])

In [ ]:
def spatial_softmax(input,  name):
    """Softmax on spatial input"""
    batch_size, num_rows, num_cols, num_channels = [d.value for d in input.shape]
    if batch_size is None:
        batch_size = -1
    
    x_map = np.empty([num_rows, num_cols], np.float32)
    y_map = np.empty([num_rows, num_cols], np.float32)

    for i in range(num_rows):
        for j in range(num_cols):
            x_map[i, j] = (i - num_rows / 2.0) / num_rows
            y_map[i, j] = (j - num_cols / 2.0) / num_cols

    x_map = tf.convert_to_tensor(x_map)
    y_map = tf.convert_to_tensor(y_map)

    x_map = tf.reshape(x_map, [num_rows * num_cols])
    y_map = tf.reshape(y_map, [num_rows * num_cols])

    features = tf.reshape(tf.transpose(net.layers['l_conv2_2'], [0,3,1,2]),
                              [batch_size, num_channels, num_rows*num_cols])
    softmax = tf.nn.softmax(features)

    fp_x = tf.reduce_sum(tf.multiply(x_map, softmax), 2)
    fp_y = tf.reduce_sum(tf.multiply(y_map, softmax), 2)

    fp = tf.reshape(tf.concat(axis=1,values=[fp_x, fp_y]), [batch_size, num_channels*2], name=name)
    return fp

In [ ]:
smax = spatial_softmax(net.layers['l_conv2_2'], 'smax')

In [ ]:
sm = sess.run([smax], {net.left_img:im.reshape(1,224,224,3)})

In [ ]:
dep_data = cv2.resize(dep, (112, 112))

In [ ]:
dep_pl

In [ ]:
softmax.shape, dep_pl.shape

In [ ]:
# dep_pl = tf.placeholder(shape=(None, 112*112), dtype=tf.float32)
tf.reduce_sum((softmax*dep_pl), axis=-1).shape

In [ ]:
tf.reshape(softmax, shape=(None, 128, 112, 112))

In [ ]:
softmax.shape

In [ ]:
batch_size, num_rows, num_cols, num_channels =[x.value for x in net.layers['l_conv2_2'].shape]

In [ ]:
batch_size

In [ ]:
num_rows.value

In [ ]:
from tftools import data
import sys
saver = None
episode_model_save_location = '/data/robotics/models/auto_episodic'

sys.path.append(pyrobo_path + '/network_models/')
sys.path.append(pyrobo_path + '../pytools/src/')
from multicam_net import PurchasePredNetOffice, PurchasePredNetHome

In [ ]:
import multicam_net
reload(multicam_net)
from multicam_net import PurchasePredNetOffice

In [ ]:
sess=tf.Session()

In [ ]:
net = PurchasePredNetOffice()


In [ ]:
dt = cad.dtype


In [ ]:
sess.run(tf.global_variables_initializer())

In [ ]:
net = PurchasePredNetHome()

In [ ]:
net.load('/data/image/models/googlenet.npy', sess, True)

In [ ]:
gnet = np.load('/data/image/models/googlenet.npy').item()

In [ ]:
net.layers['l_conv3'].shape

In [ ]:
import tensorflow as tf

In [ ]:
piter = tf.python_io.tf_record_iterator('/data/robotics/auto_episodic_data/episode_3.tfrecords')

In [ ]:
ex = tf.train.Example()

In [ ]:
dat = ex.ParseFromString(piter.next())

In [ ]:
ftr = ex.features.feature

In [ ]:
class Test:
    def test(self):
        print self.abc()
class T2(Test):
    def abc(self):
        return "s"

In [ ]:
T2().test()

### Test

In [1]:
from operator import mul
import urllib2
import numpy as np
import tensorflow as tf
import os
import cv2

In [3]:
from tftools import data
import sys
saver = None
episode_model_save_location = '/data/robotics/models/auto_episodic'
episode_save_location = '/data/robotics/auto_episodic_data/'
pyrobo_path = '/Users/kk/code/pyrobo/'

sys.path.append(pyrobo_path + '/network_models/')
sys.path.append(pyrobo_path + '../pytools/src/')


In [4]:
import trainer
import ps3_control

sess = tf.Session()

with open(episode_save_location + '/episode', 'r') as f:
    current_episode = int(f.readline().strip())


# reload(trainer)
tr = trainer.OfficeTrainer(sess, episode_model_save_location,
                          episode_save_location, current_episode)

robot_url = 'http://robot.staples.com:8888/robot/%d/%d/%d/%d/%d/%d/' 
ps3_url = 'http://solar-2.staples.com:8888/get' 
controller = ps3_control.PS3_Control(robot_url, ps3_url)

INFO:tensorflow:Restoring parameters from /data/robotics/models/auto_episodic/episode_model_4


In [8]:
controller.get_ps3_out()

[]